In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy as sp
from tqdm import tqdm
from gensim.models import KeyedVectors, Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import random
random.seed(5)
import sys
sys.path.append('../src')
from models import get_matrices_from_term_lists, \
    save_pickle, open_pickle, \
    save_experiment_arbitrary_label

In [10]:
MODEL_NAME = 'glove_840B'
SCALERS_FILEPATH = f'../data/processed/{MODEL_NAME}_scalers.pickle'
RESULTS_FILEPATH = f'../data/interim/{MODEL_NAME}_association_metric_exps.pickle'
we_model = KeyedVectors.load('../data/interim/glove_840_norm', mmap='r')

"\n# Caliskan GloVe\nglove_file = '../data/external/glove.6B/glove.6B.50d.txt'\n_ = glove2word2vec(glove_file, '../data/interim/tmp.txt')\nwe_model = KeyedVectors.load_word2vec_format('../data/interim/tmp.txt')\nprint('loading done!')\nprint(f'Total words: {len(we_model.wv.vocab)}')\n"

# Generating Distributions

In [6]:
# Fastest version, 10000 words -> 1 minute
# (Possible TODO) May be able to add minimal speedup with itemgetter 
# (see https://stackoverflow.com/questions/18453566/python-dictionary-get-list-of-values-for-list-of-keys)
# to speed up creation of word matrices in get_matrices_from_term_lists
def get_test_stat(wv_obj, X_terms, Y_terms, A_terms, B_terms):  
    [X_mtx, Y_mtx, A_mtx, B_mtx] = get_matrices_from_term_lists(we_model, X_terms, Y_terms, A_terms, B_terms)
    cosine_sim_XA = cosine_similarity(X_mtx, A_mtx)
    cosine_sim_XB = cosine_similarity(X_mtx, B_mtx)
    mean_over_Xa = np.mean(cosine_sim_XA, axis=1)
    mean_over_Xb = np.mean(cosine_sim_XB, axis=1)
    s_for_X_words = mean_over_Xa - mean_over_Xb
    # shape is (24,) or (|X_terms|,)

    cosine_sim_YA = cosine_similarity(Y_mtx, A_mtx)
    cosine_sim_YB = cosine_similarity(Y_mtx, B_mtx)
    mean_over_Ya = np.mean(cosine_sim_YA, axis=1)
    mean_over_Yb = np.mean(cosine_sim_YB, axis=1)
    s_for_Y_words = mean_over_Ya - mean_over_Yb
    test_stat = np.mean(s_for_X_words) - np.mean(s_for_Y_words)
    return test_stat
get_test_stat(we_model, X_terms, Y_terms, A_terms, B_terms)

0.090876885

In [6]:
# This cell works too. It takes twice as long as the cell above,
# but if we want to try to vectorize the outer loop, then 
# we will probably have to use this version
def calculate_association_metric_for_target_word(word_vec, A_mtx, B_mtx):
    '''Computes the association metric, s(w,A,B).
    word_vec: 1-D word vector
    A_mtx, B_mtx: 2-D word vector arrays'''
    A_dot_v = np.dot(A_mtx, word_vec)
    B_dot_v = np.dot(B_mtx, word_vec)
    A_norms = np.multiply(np.linalg.norm(A_mtx, axis=1), np.linalg.norm(word_vec))
    B_norms = np.multiply(np.linalg.norm(B_mtx, axis=1), np.linalg.norm(word_vec))
    A_cosines = np.divide(A_dot_v, A_norms)
    B_cosines = np.divide(B_dot_v, B_norms)
    return np.mean(A_cosines) - np.mean(B_cosines)

for i in tqdm(range(10000)):
    [X_mtx, Y_mtx, A_mtx, B_mtx] = get_matrices_from_term_lists(we_model, X_terms, Y_terms, A_terms, B_terms)
    X_associations = np.apply_along_axis(lambda x_vec: calculate_association_metric_for_target_word(x_vec, A_mtx, B_mtx), 1, X_mtx)
    Y_associations = np.apply_along_axis(lambda y_vec: calculate_association_metric_for_target_word(y_vec, A_mtx, B_mtx), 1, Y_mtx)
    m = np.mean(X_associations) - np.mean(Y_associations)


100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:50<00:00, 90.22it/s]


In [14]:
def get_n_test_stats(wv_obj, X_terms, Y_terms, A_terms, B_terms, n_samples=100):
    sigtest_dist_1 = []
    sigtest_dist_2 = []
    sigtest_dist_3 = []
    n_targets = len(X_terms)
    n_attributes = len(A_terms)
    assert len(X_terms) == len(Y_terms)
    assert len(A_terms) == len(B_terms)
    vocab_list = list(wv_obj.wv.vocab)
    random.seed(5)
    for i in tqdm(range(n_samples)):
        X_sample = random.sample(vocab_list, k=n_targets)
        Y_sample = random.sample(vocab_list, k=n_targets)
        sigtest_dist_1.append(get_test_stat(wv_obj, X_sample, Y_sample, A_terms, B_terms))
        sigtest_dist_2.append(get_test_stat(wv_obj, X_terms, Y_sample, A_terms, B_terms))
        sigtest_dist_3.append(get_test_stat(wv_obj, Y_terms, X_sample, A_terms, B_terms))
        #sigtest_dist_3.append(get_test_stat(wv_obj, X_sample, Y_terms, A_terms, B_terms))
    return np.array(sigtest_dist_1), np.array(sigtest_dist_2), np.array(sigtest_dist_3)
#a,b,c = get_n_test_stats(we_model, X_terms, Y_terms, A_terms, B_terms)

In [15]:
# Reminder that if you run this cell with a lower number of n_samples, 
# It will overwrite what's currently in the dictionary
FILEPATH = '../data/interim/glove_840B_association_metric_exps.pickle'
EXPERIMENT_DEFINITION_PATH = '../data/interim/glove_840B_experiment_definitions.pickle'
'''
FILEPATH = '../data/interim/association_metric_exps.pickle'
EXPERIMENT_DEFINITION_PATH = '../data/interim/experiment_definitions.pickle'
'''
def run_all_sigtests(order='second'):
    #TODO: refactor: remove 'order' argument, no longer necessary
    exps = open_pickle(EXPERIMENT_DEFINITION_PATH)
    scalers_dict = open_pickle(SCALERS_FILEPATH)
    print(f'ORDER = {order}')
    for exp_num, exp in exps.items():
        print('******************************')
        print(f'Experiment: {exp_num}')
        X_terms = exp['X_terms']
        Y_terms = exp['Y_terms']
        A_terms = exp['A_terms']
        B_terms = exp['B_terms']
        #scaler = scalers_dict[exp_num][order]
        comparison_statistic = get_test_stat(we_model, X_terms, Y_terms, A_terms, B_terms)
        dist_1, dist_2, dist_3 = get_n_test_stats(we_model, X_terms, Y_terms, A_terms, B_terms, n_samples=10000)
        #[dist_1, dist_2, dist_3] = [scaler.transform(dist.reshape(-1,1)).reshape(len(dist)) for dist in [dist_1, dist_2, dist_3]]
        save_experiment_arbitrary_label(FILEPATH, exp_num, order,
                                        'sigtest_dist_1', dist_1)
        save_experiment_arbitrary_label(FILEPATH, exp_num, order,
                                        'sigtest_dist_2', dist_2)
        save_experiment_arbitrary_label(FILEPATH, exp_num, order,
                                        'sigtest_dist_3', dist_3)
run_all_sigtests(order='second')

ORDER = second
******************************
Experiment: 1


C:\Users\danie\Miniconda3\envs\semproject2\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':
 19%|██████████████▊                                                              | 1916/10000 [00:44<03:09, 42.77it/s]


KeyboardInterrupt: 

# Fitting Distributions

In [3]:
RESULTS_FILEPATH = '../data/interim/glove_840B_association_metric_exps.pickle'
d = open_pickle(RESULTS_FILEPATH)

In [4]:
d[1]

defaultdict(dict,
            {'second': {'X_array': array([ 0.04218954,  0.07744247, -0.00571454,  0.05228046,  0.04121482,
                      0.01642609,  0.05941069,  0.08293772,  0.10962468,  0.06767094,
                      0.05554709,  0.08660334,  0.11914104,  0.02486253,  0.09582818,
                      0.06356439,  0.10746822,  0.09457076,  0.0782218 ,  0.06955409,
                      0.07021624,  0.00056103,  0.07109076,  0.00462493,  0.01719075],
                    dtype=float32),
              'Y_array': array([-0.06156641, -0.02580431, -0.01672459, -0.09525567, -0.04476053,
                     -0.16175112, -0.07126629,  0.05551386, -0.14847207, -0.04637542,
                      0.02886164, -0.11517835, -0.08661395, -0.07366097, -0.09728244,
                     -0.07608503,  0.02309245, -0.10237056, -0.07108605, -0.11936021,
                     -0.1044682 ,  0.08603638, -0.07104123, -0.0895586 , -0.11381534],
                    dtype=float32),
              'X

In [16]:
from scipy.stats import norm
'''
FILEPATH = '../data/interim/association_metric_exps.pickle'
EXPERIMENT_DEFINITION_PATH = '../data/interim/experiment_definitions.pickle'
'''
FILEPATH = '../data/interim/glove_840B_association_metric_exps.pickle'
EXPERIMENT_DEFINITION_PATH = '../data/interim/glove_840B_experiment_definitions.pickle'
def calculate_all_sigtest_metrics(order='second'):
    #TODO: refactor: remove 'order' argument, no longer necessary
    exps = open_pickle(EXPERIMENT_DEFINITION_PATH)
    scalers_dict = open_pickle(SCALERS_FILEPATH)
    results_dict = open_pickle(RESULTS_FILEPATH)
    print(f'ORDER = {order}')
    for exp_num, exp in exps.items():
        print('******************************')
        print(f'Experiment: {exp_num}')
        X_terms = exp['X_terms']
        Y_terms = exp['Y_terms']
        A_terms = exp['A_terms']
        B_terms = exp['B_terms']
        comparison_statistic = get_test_stat(we_model, X_terms, Y_terms, A_terms, B_terms)

        dist_1, dist_2, dist_3 = [results_dict[exp_num][order][f'sigtest_dist_{n}'] for n in [1,2,3]]
        loc_1, loc_2, loc_3 = [np.mean(dist) for dist in [dist_1, dist_2, dist_3]]
        scale_1, scale_2, scale_3 = [np.std(dist) for dist in [dist_1, dist_2, dist_3]]
        # If you want to play around with statistics of the distributions,
        # Add code and print statements here, e.g.
        # print(f'90% CI for dist 1: {norm.ppf(0.1, loc=loc_1, scale=scale_1)}')

        save_experiment_arbitrary_label(FILEPATH, exp_num, order, 'test_statistic', comparison_statistic)
        save_experiment_arbitrary_label(FILEPATH, exp_num, order,
                                        'ST1_p-value', norm.cdf(comparison_statistic, loc=loc_1, scale=scale_1))

calculate_all_sigtest_metrics(order='second')

ORDER = second
******************************
Experiment: 1
Results array successfully saved to file ../data/interim/glove_840B_association_metric_exps.pickle under    keys [1][second][test_statistic]
Results array successfully saved to file ../data/interim/glove_840B_association_metric_exps.pickle under    keys [1][second][ST1_p-value]
******************************
Experiment: 2
Results array successfully saved to file ../data/interim/glove_840B_association_metric_exps.pickle under    keys [2][second][test_statistic]
Results array successfully saved to file ../data/interim/glove_840B_association_metric_exps.pickle under    keys [2][second][ST1_p-value]
******************************
Experiment: 3
Results array successfully saved to file ../data/interim/glove_840B_association_metric_exps.pickle under    keys [3][second][test_statistic]
Results array successfully saved to file ../data/interim/glove_840B_association_metric_exps.pickle under    keys [3][second][ST1_p-value]
*************